In [3]:
import h5py
import os
import soundcard as sc
import soundfile as sf
import sounddevice as sd
import librosa
import numpy as np
from keras.models import load_model
from keras.models import load_model
# import test2
# import make_dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam
import os
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
from datetime import datetime
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd
# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa
from sklearn.decomposition import PCA
# Visualization
import matplotlib.pyplot as plt
#import seaborn as sns
import librosa.display
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd
from plotly.subplots import make_subplots
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile



2023-10-06 15:16:03.653125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 15:16:03.768723: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-06 15:16:03.768740: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-06 15:16:03.791725: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-06 15:16:04.292494: W tensorflow/stream_executor/pla

In [4]:

def record_and_save_clips(record_time):
    output_path = r'E:/job/audio/recordings'
    
    headphones_device_id = "Headphones (Realtek(R) Audio)"
   

    try:
            output_file_name = f"{output_path}/out_{clip_count}.wav"
            samplerate = 16000
            with sc.get_microphone(id=headphones_device_id, include_loopback=True).recorder(samplerate=samplerate) as mic:
                
                data = mic.record(numframes=samplerate * record_time)
                sf.write(file=output_file_name, data=data[0:, 0], samplerate=samplerate)
                print("Saved successfully " + str(clip_count))
                

    except Exception as e:
            print("An error occurred:", str(e))
#it will save 1 clip


def extract_feature(base_path, clip_count, max_pad_length=86):
    # Combine the base path and variable part to form the complete file path
    file_name = f"/home/sae/saevoice.wav"
    
    try:
            audio, sample_rate = librosa.load(file_name)
            mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
            pad_width = max_pad_length - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
    except Exception as e:
            print("Error encountered while parsing file: ", file_name)
            return None
    return mfccs
#it will process the same clip into mfcc

def predict(mfccs):
      
      h5_file_path = r'E:/job/audio/for2_mfcc.h5'
      loaded_model = load_model(h5_file_path)
      new_predictions=loaded_model.predict(mfccs)
      # Load the existing CSV file with previous predictions
      predict_path=r'E:/job/audio/predictions.csv'
      previous_predictions_df = pd.read_csv(predict_path)
      # Create a new DataFrame for the new predictions
      new_predictions_df = pd.DataFrame(new_predictions, columns=['New_Prediction'])

      # Concatenate the previous and new predictions horizontally (along columns)
      combined_predictions_df = pd.concat([previous_predictions_df, new_predictions_df], axis=1)

      # Save the combined DataFrame to the same CSV file
      combined_predictions_df.to_csv('predictions.csv', index=False)

#it will predict the result and save it in a csv file 





In [5]:
number_of_clips = int(input("Enter the number of clips to record: "))
record_time = int(input("Enter the duration of each clip (in seconds): "))   
clip_count = 1
#the function will run as long as each clip has not been predicted 
# input("Press enter to start recording....")
record_and_save_clips(record_time)
#take 65 seconds only
#and number of recordings as 1 right now we will add the while loop later

ValueError: invalid literal for int() with base 10: ''

In [6]:
mfcc_data=extract_feature(r'E:/job/audio/recordings',1)
print("feature extracted")
print("MFCC shape:", mfcc_data.shape) 


feature extracted
MFCC shape: (40, 2845)


In [7]:
input_data=mfcc_data
print("MFCC shape:", mfcc_data.shape) 

MFCC shape: (40, 2845)


In [8]:
desired_shape = (40, 2784)

if mfcc_data.shape[1] > desired_shape[1]:
    # Slice the data to fit the desired shape
    truncated_data = mfcc_data[:, :desired_shape[1]]
else:
    print("Cannot truncate. Desired shape is larger or equal to the original shape.")

In [28]:
truncated_data.shape

(40, 2784)

In [9]:
desired_shape = (32, 40, 87)

# Check if the shapes are compatible
if np.prod(truncated_data.shape) != np.prod(desired_shape):
    print("Cannot reshape. Total number of elements must remain the same.")
else:
    reshaped_data = truncated_data.reshape(desired_shape)

In [12]:
h5_file_path = r'model1.h5'
loaded_model = load_model(h5_file_path)
print("model is loaded")
new_predictions=loaded_model.predict(reshaped_data)
print(new_predictions)
#0 is for fake
#1 is for real

2023-10-06 15:28:45.731814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-06 15:28:45.731947: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-10-06 15:28:45.732032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-10-06 15:28:45.732115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-10-06 15:28:45.732197: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

model is loaded
1/1 [==============================] - 0s 369ms/step
[[2.2873646e-03 9.9771255e-01]
 [1.0796006e-08 9.9999994e-01]
 [2.6849676e-08 9.9999994e-01]
 [2.1977413e-03 9.9780226e-01]
 [9.3579304e-01 6.4207025e-02]
 [8.5648757e-01 1.4351243e-01]
 [4.3285675e-02 9.5671433e-01]
 [9.8051274e-01 1.9487260e-02]
 [9.9994546e-01 5.4495289e-05]
 [2.2047525e-05 9.9997789e-01]
 [6.9536828e-03 9.9304634e-01]
 [7.7232534e-01 2.2767462e-01]
 [9.0649241e-01 9.3507491e-02]
 [1.9405162e-02 9.8059481e-01]
 [8.7992549e-03 9.9120063e-01]
 [4.9097589e-01 5.0902408e-01]
 [6.2045287e-02 9.3795460e-01]
 [6.2794872e-02 9.3720508e-01]
 [9.9007279e-01 9.9272626e-03]
 [1.7597207e-01 8.2402796e-01]
 [9.9986911e-01 1.3085250e-04]
 [9.6944338e-01 3.0556628e-02]
 [8.7548369e-01 1.2451623e-01]
 [2.6902657e-03 9.9730980e-01]
 [2.0903128e-05 9.9997908e-01]
 [5.9522954e-08 9.9999994e-01]
 [3.7529138e-05 9.9996239e-01]
 [5.0344585e-08 9.9999994e-01]
 [6.1512645e-07 9.9999934e-01]
 [1.6730425e-09 9.9999994e-01]
 

In [13]:
threshold = 0.7
binary_predictions = [1 if prob[1] >= threshold else 0 for prob in new_predictions]

In [17]:
binary_predictions= (new_predictions[:, 1] > 0.7).astype(int)


In [18]:
print( binary_predictions)

[1 1 1 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 1 0 1]


In [19]:
avg_prediction = sum(binary_predictions) / len(binary_predictions)
print(avg_prediction)

0.625
